In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
COLS_X = ['AP','P','AI','I']
COLS_Y = ['Y']

In [3]:
def random_logit(x):
    z = 1./(1+np.exp(-x))
    s = np.random.binomial(n=1, p=z)
    return s
    

def gen_synth_data(N=10000, verbose=False):
    np.random.seed(10)
    '''Example 4 from main text
    Variables
    A_P: Neighborhood_SES as low or high 0 or 1
    A_I: Individual_SES as low or high 0 or 1
    P: population_level attributes 
    I: individual_level attributes
    Y: health status 0 or 1'''
    
    # Regression coefficients
    # gamma_PAP = 1
    # gamma_AIAP = 3
    # gamma_AIP = 2
    # gamma_IAP = 8
    # gamma_IAI = 10
    # gamma_YP = 1
    # gamma_YI = 1
    # gamma_YAI = 2
    #zero ap effect
    gamma_PAP = 1
    gamma_AIAP = 3
    gamma_AIP = 2
    gamma_IAP = 8
    gamma_IAI = 10
    gamma_YP = 1
    gamma_YI = 1
    gamma_YAI = 2
    scale_e = 0.64
    
    AP = random_logit(np.random.uniform(0,1, size=N))
    P  = gamma_PAP*AP + 0.001*np.random.normal(loc=0.0, scale=scale_e , size=N)
    AI = random_logit(gamma_AIAP*AP + gamma_AIP*P + 0.001*np.random.normal(loc=0.0, scale=0.1, size=N))
    I  = gamma_IAP*AP + gamma_IAI*AI + 0.001*np.random.normal(loc=0.0, scale=scale_e, size=N)
    # Y  = random_logit(gamma_YP*P + gamma_YI*I + gamma_YAI*AI+ 0.001*np.random.normal(loc=0.0, scale=scale_e, size=N))
    Y  = gamma_YP*P + gamma_YI*I + gamma_YAI*AI+ 0.001*np.random.normal(loc=0.0, scale=scale_e, size=N)
    data_Y = Y
    data_X = np.stack([AP,P,AI,I],axis=1)
    cols_X = ['AP','P','AI','I']
    cols_Y = ['Y']
    X_train, X_test, Y_train, Y_test = train_test_split(data_X, data_Y, test_size=0.25, random_state=42)
    X_train = pd.DataFrame(data = X_train, columns = cols_X)
    X_test  = pd.DataFrame(data = X_test, columns = cols_X)
    Y_train = pd.DataFrame(data = Y_train, columns = cols_Y)
    Y_test  = pd.DataFrame(data = Y_test, columns = cols_Y)
    return X_train, X_test, Y_train, Y_test

In [4]:
X_train, X_test, Y_train, Y_test = gen_synth_data()

In [5]:
def fit_models(X_train, Y_train, formula_P, formula_AI, formula_I, formula_Y):
    data = pd.concat([X_train,Y_train],axis=1)
    # print(data.head())
    
    # cols_X = ['AP','P','AI','I']
    # cols_Y = ['Y']
    # #P ~ AP
    model_P = LinearRegression(fit_intercept=True)
    model_P.fit(data[['AP']],data[['P']])
    beta_P = model_P.coef_
   

    # # AI ~ P+AP
    model_AI = LogisticRegression(penalty='l2',solver='saga', fit_intercept=True)
    model_AI.fit(data[['P','AP']],data[['AI']])
    beta_AI = model_AI.coef_
   
    # # I ~ AP + AI
    model_I = LinearRegression(fit_intercept=True)
    model_I.fit(data[['AP','AI']],data[['I']])
    beta_I = model_I.coef_
  

    # # Y ~ P + I + AI
    # model_Y = LogisticRegression(penalty='l2',solver='saga', fit_intercept=True)
    model_Y = LinearRegression(fit_intercept=True)

    model_y = model_Y.fit(data[['P','I','AI']],data[['Y']])
    beta_Y = model_Y.coef_
    
    return model_Y,{'P':beta_P, 'AI':beta_AI, 'I':beta_I, 'Y':beta_Y}


def classifiers_all(X_train,Y_train,cols_X=COLS_X,cols_Y=COLS_Y):
    #formulas for the different classifiers:
    #P vs AP
    formula_P = 'P ~ AP'

    #AI vs AP,P
    formula_AI = 'AI ~ AP + P'

    #I vs AP,AI
    formula_I = 'I ~ AP + AI'

    #Y vs P,I,AI
    formula_Y = 'Y ~ P + I + AI'

    model_Y, beta_lm = fit_models(X_train,Y_train, formula_P, formula_AI, formula_I, formula_Y)

    return model_Y,beta_lm
model_Y,beta_lm = classifiers_all(X_train,Y_train)


In [6]:
#calculate path-specific effects
def pse(beta_lm):
    theta_y = beta_lm['Y'][0]
    theta_i = beta_lm['I'][0]
    theta_ai = beta_lm['AI'][0]
    theta_p = beta_lm['P'][0]

    pse_ai = {'ai_y': theta_y[2],\
              'ai_i_y':theta_y[1]*theta_i[1] ,\
              'ai__y': theta_y[2]+theta_y[1]*theta_i[1] }

    pse_ap_ai = {'ap_p_y,ai_y': theta_y[0]*theta_p[0]+theta_y[2],\
                'ap_p_y,ai_i_y': theta_y[0]*theta_p[0]+theta_y[1]*theta_i[1],\
                'ap_p_y,ai_i_y,ai_y': theta_y[0]*theta_p[0]+theta_y[1]*theta_i[1]+theta_y[2],\
                'ap_i_y,ai_i_y': theta_y[1] *(theta_i[1] + theta_i[0]),\
                'ap_i_y,ai_y':theta_y[2] + theta_y[1]*theta_i[0],\
                'ap_i_y,ai__y':theta_y[2] + theta_y[1] *(theta_i[1] + theta_i[0]),\
                'ap__y,ai__y':theta_y[2] + theta_y[1] *(theta_i[1] + theta_i[0])+theta_y[1]*theta_i[0]}
    return pse_ai,pse_ap_ai



In [7]:
pse_ai,pse_ai_ap = pse(beta_lm)
print(pse_ai)
print(pse_ai_ap)


{'ai_y': 2.034593479286903, 'ai_i_y': 9.965387176764633, 'ai__y': 11.999980656051536}
{'ap_p_y,ai_y': 3.0622841688726945, 'ap_p_y,ai_i_y': 10.993077866350424, 'ap_p_y,ai_i_y,ai_y': 13.027671345637327, 'ap_i_y,ai_i_y': 17.937681587713826, 'ap_i_y,ai_y': 10.006887890236094, 'ap_i_y,ai__y': 19.97227506700073, 'ap__y,ai__y': 27.94456947794992}
